<a href="https://colab.research.google.com/github/AleAguirreM/Analitica3AA/blob/main/c_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
from sqlalchemy import create_engine
from mlxtend.preprocessing import TransactionEncoder

In [2]:
conn = sql.connect("/content/drive/MyDrive/db_moviesSQL")

In [3]:
curr = conn.cursor()

In [4]:
curr.execute("SELECT name FROM sqlite_master WHERE type = 'table'")

In [5]:
curr.fetchall()

[('ratings',),
 ('movies',),
 ('full',),
 ('reco',),
 ('movies_sel',),
 ('movies_n',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',)]

In [6]:
ratings = pd.read_sql("select * from ratings", conn)

In [7]:
pd.read_sql("select * from movies", conn)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [8]:

full_ratings = pd.read_sql("select * from full_ratings", conn)
#el id esta repetido, lo eliminamos
full_ratings = full_ratings.drop(columns=['movieId:1', 'movieId:2'])
full_ratings

,userId,movieId,rating,timestamp,title,genres,cnt_rat
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,52
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,102
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,203
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,204
...,...,...,...,...,...,...,...
66940,610,148626,4.0,1493847175,"Big Short, The (2015)",Drama,26
66941,610,152081,4.0,1493846503,Zootopia (2016),Action|Adventure|Animation|Children|Comedy,32
66942,610,164179,5.0,1493845631,Arrival (2016),Sci-Fi,26
66943,610,166528,4.0,1493879365,Rogue One: A Star Wars Story (2016),Action|Adventure|Fantasy|Sci-Fi,27


In [9]:
### cambiamos el formato de fecha para poder trabjar con fecha
full_ratings['timestamp'] = pd.to_datetime(full_ratings['timestamp'], unit='s')
## cambiamos el nombre de la columna
full = full_ratings.rename(columns={'timestamp': 'date'})

## extraemos el año
full['year'] = full['date'].dt.year
full

,userId,movieId,rating,date,title,genres,cnt_rat,year
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215,2000
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,52,2000
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,102,2000
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,203,2000
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,204,2000
...,...,...,...,...,...,...,...,...
66940,610,148626,4.0,2017-05-03 21:32:55,"Big Short, The (2015)",Drama,26,2017
66941,610,152081,4.0,2017-05-03 21:21:43,Zootopia (2016),Action|Adventure|Animation|Children|Comedy,32,2017
66942,610,164179,5.0,2017-05-03 21:07:11,Arrival (2016),Sci-Fi,26,2017
66943,610,166528,4.0,2017-05-04 06:29:25,Rogue One: A Star Wars Story (2016),Action|Adventure|Fantasy|Sci-Fi,27,2017


In [10]:
###### Para separar géneros en base de datos
genres=full['genres'].str.split('|')
te = TransactionEncoder()
genres = te.fit_transform(genres)
genres = pd.DataFrame(genres, columns = te.columns_)
genres  = genres.astype(int)
movies = full.drop(['genres'], axis = 1)
full =pd.concat([movies, genres],axis=1)

In [11]:
full

,userId,movieId,rating,date,title,cnt_rat,year,Action,Adventure,Animation,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),215,2000,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),52,2000,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),102,2000,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),203,2000,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",204,2000,0,0,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66940,610,148626,4.0,2017-05-03 21:32:55,"Big Short, The (2015)",26,2017,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66941,610,152081,4.0,2017-05-03 21:21:43,Zootopia (2016),32,2017,1,1,1,...,0,0,0,0,0,0,0,0,0,0
66942,610,164179,5.0,2017-05-03 21:07:11,Arrival (2016),26,2017,0,0,0,...,0,0,0,0,0,0,1,0,0,0
66943,610,166528,4.0,2017-05-04 06:29:25,Rogue One: A Star Wars Story (2016),27,2017,1,1,0,...,0,0,0,0,0,0,1,0,0,0


In [12]:
##Eenvimos la tabla modificada a SQL para actualizar cambios
full.to_sql("full",conn,if_exists="replace")


66945

In [13]:
pd.read_sql("select * from full", conn)

,index,userId,movieId,rating,date,title,cnt_rat,year,Action,Adventure,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),215,2000,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),52,2000,0,0,...,0,0,0,0,0,1,0,0,0,0
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),102,2000,1,0,...,0,0,0,0,0,0,0,1,0,0
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),203,2000,0,0,...,0,0,0,0,1,0,0,1,0,0
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",204,2000,0,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66940,66940,610,148626,4.0,2017-05-03 21:32:55,"Big Short, The (2015)",26,2017,0,0,...,0,0,0,0,0,0,0,0,0,0
66941,66941,610,152081,4.0,2017-05-03 21:21:43,Zootopia (2016),32,2017,1,1,...,0,0,0,0,0,0,0,0,0,0
66942,66942,610,164179,5.0,2017-05-03 21:07:11,Arrival (2016),26,2017,0,0,...,0,0,0,0,0,0,1,0,0,0
66943,66943,610,166528,4.0,2017-05-04 06:29:25,Rogue One: A Star Wars Story (2016),27,2017,1,1,...,0,0,0,0,0,0,1,0,0,0


In [70]:
###crear una tabla que contenga solo una vez el nombre de las peliculas
x = pd.read_sql("""SELECT
    b.movieId as movieId,
    b.genres as genres,
    AVG(a.rating) as rating
FROM
    movies b
INNER JOIN
    full a ON b.movieId = a.movieId
GROUP BY
    b.title;
            """, conn)
x

,movieId,genres,rating
0,69757,Comedy|Drama|Romance,3.666667
1,2572,Comedy|Romance,3.527778
2,1367,Adventure|Children|Comedy,3.074468
3,2085,Adventure|Animation|Children,3.431818
4,1203,Drama,4.149123
...,...,...,...
1289,4816,Comedy,3.509259
1290,152081,Action|Adventure|Animation|Children|Comedy,3.890625
1291,2600,Action|Sci-Fi|Thriller,3.863636
1292,5507,Action|Crime|Thriller,2.770833


In [58]:
##Eenvimos la tabla modificada a SQL para actualizar cambios
x.to_sql("x",conn,if_exists="replace")

9737

In [78]:
###### Para separar géneros en base de datos y crear tabla dumisada sin duplicados
a = x['genres'].str.split('|')
te = TransactionEncoder()
a = te.fit_transform(a)
a = pd.DataFrame(a, columns = te.columns_)
a  = a.astype(int)
b = x.drop(['genres'], axis = 1)
dumm2 =pd.concat([b, a],axis=1)


In [14]:
######################################################################
################## 1. sistemas basados en popularidad ###############
#####################################################################


##### recomendaciones basado en popularidad ######

#### mejores calificadas que tengan calificacion
pd.read_sql("""select title,
            avg(rating) as avg_rat,
            count(*) as read_num
            from full
            where rating<>0
            group by title
            order by avg_rat desc
            limit 10
            """, conn)

,title,avg_rat,read_num
0,"Streetcar Named Desire, A (1951)",4.475000,20
1,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.333333,27
2,"Philadelphia Story, The (1940)",4.310345,29
3,Lawrence of Arabia (1962),4.300000,45
4,In the Name of the Father (1993),4.300000,25
5,Hoop Dreams (1994),4.293103,29
6,"Godfather, The (1972)",4.289062,192
7,Harold and Maude (1971),4.288462,26
8,Logan (2017),4.280000,25
9,Fight Club (1999),4.272936,218


In [15]:
#### las peliculas mas vistas con promedio de los que calficaron ###
pd.read_sql("""select title,
            avg(iif(rating = 0, Null, rating)) as avg_rat,
            count(*) as read_num
            from full
            group by title
            order by read_num desc
            """, conn)

,title,avg_rat,read_num
0,"Silence of the Lambs, The (1991)",4.161290,279
1,"Matrix, The (1999)",4.192446,278
2,Star Wars: Episode IV - A New Hope (1977),4.231076,251
3,Jurassic Park (1993),3.750000,238
4,Braveheart (1995),4.031646,237
...,...,...,...
1289,"Animatrix, The (2003)",3.700000,20
1290,Anastasia (1997),3.350000,20
1291,Adventures in Babysitting (1987),3.325000,20
1292,"6th Day, The (2000)",2.800000,20


In [16]:

#######################################################################
######## 2.1 Sistema de recomendación basado en contenido un solo producto - Manual ########
#######################################################################

movies1 = pd.read_sql('select * from full',conn)
movies1.info()
movies1['year']=movies1.year.astype('int')
movies1.info()
##### escalar para que año esté en el mismo rango ###
sc=MinMaxScaler()
movies1[["year_sc"]]=sc.fit_transform(movies1[['year']])
movies1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66945 entries, 0 to 66944
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        66945 non-null  int64  
 1   userId       66945 non-null  int64  
 2   movieId      66945 non-null  int64  
 3   rating       66945 non-null  float64
 4   date         66945 non-null  object 
 5   title        66945 non-null  object 
 6   cnt_rat      66945 non-null  int64  
 7   year         66945 non-null  int64  
 8   Action       66945 non-null  int64  
 9   Adventure    66945 non-null  int64  
 10  Animation    66945 non-null  int64  
 11  Children     66945 non-null  int64  
 12  Comedy       66945 non-null  int64  
 13  Crime        66945 non-null  int64  
 14  Documentary  66945 non-null  int64  
 15  Drama        66945 non-null  int64  
 16  Fantasy      66945 non-null  int64  
 17  Film-Noir    66945 non-null  int64  
 18  Horror       66945 non-null  int64  
 19  IMAX

In [17]:
## eliminar filas que no se van a utilizar ###

movies_dum1= movies1.drop(columns=['movieId','title','cnt_rat','year','date','userId','date','index'])

In [18]:
#### convertir a dummies


joblib.dump(movies_dum1,"salidas\\movies_dum1.joblib") ### para utilizar en segundos modelos


['salidas\\movies_dum1.joblib']

In [26]:
m = 'Apollo 13 (1995)'
ind_movie = movies1[movies1['title'] == m].index.values.astype(int)[0]

# Suponiendo que 'movies_dum1' es el DataFrame con características codificadas como dummies
similar_movies = movies_dum1.corrwith(movies_dum1.iloc[ind_movie, :], axis=1)

# Ordenar películas similares por correlación
similar_movies = similar_movies.sort_values(ascending=False)

# Excluir la película de referencia de la lista de películas similares
similar_movies = similar_movies.drop(ind_movie)

# Inicializar una lista para almacenar las películas recomendadas
recommended_movies = []

# Iterar sobre las películas similares y agregar películas únicas a la lista de recomendaciones
for movie_index in similar_movies.index:
    # Obtener el nombre de la película
    movie_title = movies1["title"].iloc[movie_index]
    # Verificar si la película ya está en la lista de recomendaciones
    if movie_title not in recommended_movies:
        # Agregar la película a la lista de recomendaciones
        recommended_movies.append(movie_title)
    # Verificar si ya hemos seleccionado suficientes recomendaciones (10 en este caso)
    if len(recommended_movies) >= 10:
        break

# Crear un DataFrame con las películas recomendadas
top_similar_movies = pd.DataFrame(recommended_movies, columns=["title"])

# Mostrar las películas más similares
top_similar_movies


,title
0,Apollo 13 (1995)
1,Life of Pi (2012)
2,Harry Potter and the Order of the Phoenix (2007)
3,Robin Hood: Prince of Thieves (1991)
4,Stand by Me (1986)
5,Easy Rider (1969)
6,"Beach, The (2000)"
7,Captain Phillips (2013)
8,"Motorcycle Diaries, The (Diarios de motociclet..."
9,Thor (2011)


In [20]:
def recomendacion(pelicula=list(full['title'])):
    # Obtener el índice de la película seleccionada de la lista
    ind_movies = full[full['title'] == pelicula].index.values.astype(int)[0]

    # Calcular la correlación entre la película seleccionada y todas las demás
    similar_movies = movies_dum1.corrwith(movies_dum1.iloc[ind_movies, :], axis=1)

    # Ordenar las correlaciones de mayor a menor
    similar_movies = similar_movies.sort_values(ascending=False)

    # Inicializar un DataFrame para almacenar las recomendaciones únicas
    top_similar_movies = pd.DataFrame(columns=["title", "correlación"])

    # Iterar sobre las películas similares y agregarlas a las recomendaciones únicas
    for movie_index, correlation in similar_movies.items():
        # Verificar si la película es diferente a la seleccionada y no está en las recomendaciones únicas
        if full["title"].iloc[movie_index] != pelicula and not (top_similar_movies["title"] == full["title"].iloc[movie_index]).any():
            # Agregar la película a las recomendaciones únicas
            top_similar_movies.loc[len(top_similar_movies)] = {
                "title": full["title"].iloc[movie_index],
                "correlación": correlation
            }

        # Verificar si ya hemos seleccionado suficientes recomendaciones (10 en este caso)
        if len(top_similar_movies) >= 10:
            break

    return top_similar_movies

print(interact(recomendacion))

interactive(children=(Dropdown(description='pelicula', options=('Toy Story (1995)', 'Grumpier Old Men (1995)',…

<function recomendacion at 0x7c8dd553dcf0>


In [80]:
#############################################################################################
#### 2.1 Sistema de recomendación basado en contenido KNN un solo producto visto #################
##############################################################################################

##### ### entrenar modelo #####

## el coseno de un angulo entre dos vectores es 1 cuando son perpendiculares y 0 cuando son paralelos(indicando que son muy similar324e-06	3.336112e-01	3.336665e-01	3.336665e-es)
model = neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
model.fit(dumm2)
dist, idlist = model.kneighbors(dumm2)


distancias=pd.DataFrame(dist) ## devuelve un ranking de la distancias más cercanas para cada fila(pelicula)
id_list=pd.DataFrame(idlist) ## para saber esas distancias a que item corresponde


In [81]:
####ejemplo para una pelicula
movies_list_name = []
movie_name = 'Apollo 13 (1995)'
movie_id = movies[movies['title'] == movie_name].index[0]  # Obtener el índice de la película

for newid in idlist[movie_id]:
    recommended_movie_name = movies.loc[newid].title
    if recommended_movie_name != movie_name:  # Verificar si el nombre no es igual al de la película original
        movies_list_name.append(recommended_movie_name)  # Agregar el nombre de la película recomendada
print(movies_list_name)

['G.I. Jane (1997)', 'Philadelphia (1993)', 'Sneakers (1992)', "Pan's Labyrinth (Laberinto del fauno, El) (2006)", '27 Dresses (2008)', 'Thing, The (1982)', 'Doors, The (1991)', 'Departed, The (2006)', 'Face/Off (1997)', 'Seven (a.k.a. Se7en) (1995)']


In [82]:
def MovieRecommender(movie_name=list(full['title'].value_counts().index)):
    movies_list_name = []
    movie_id = full[full['title'] == movie_name].index
    movie_id = movie_id[0]
    recommended_movies = set()  # Conjunto para almacenar las películas recomendadas únicas
    for newid in idlist[movie_id]:
        # Verificar si la película recomendada no es la misma que la seleccionada
        if full.loc[newid].title != movie_name:
            # Verificar si la película recomendada no ha sido recomendada anteriormente
            if full.loc[newid].title not in recommended_movies:
                movies_list_name.append(full.loc[newid].title)
                recommended_movies.add(full.loc[newid].title)  # Agregar la película recomendada al conjunto
        # Recomendar al menos 5 películas, aunque la relación sea baja
        if len(movies_list_name) >= 5:
            break
    return movies_list_name[:5]  # Devolver solo las primeras 5 películas recomendadas

print(interact(MovieRecommender))


interactive(children=(Dropdown(description='movie_name', options=('Silence of the Lambs, The (1991)', 'Matrix,…

<function MovieRecommender at 0x7c8dc99c8040>
